# Installation

In [ ]:
pip install --force-reinstall numpy pandas tensorflow tkan


In [ ]:
!pip install numpy==1.24.3
!pip install scipy==1.10.1


# Codes

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tkan import TKAN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset from det1.csv
data = pd.read_csv('det1.csv')

# Convert Date column to datetime and set as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Define the number of past days to use as input (sequence length)
sequence_length = 3

# Prepare input sequences and output values
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length].values)
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Create sequences
X, y = create_sequences(data['Temp'], sequence_length)

# Reshape X for model compatibility (samples, time steps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Define and compile the model
model = Sequential([
    TKAN(64, sub_kan_configs=[{'spline_order': 2, 'grid_size': 5}], return_sequences=False),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=100, batch_size=1, verbose=1)


In [17]:
# Function to predict temperature for a given date
def predict_temp_for_date(input_date, data, model, sequence_length=3):
    # Convert input_date to datetime
    input_date = pd.to_datetime(input_date)
    
    # Check if input_date is beyond the training data
    last_date = data.index[-1]
    if input_date <= last_date:
        print(f"Date {input_date.date()} is within the training range. Prediction not applicable for past dates.")
        return None

    # Generate last sequence from the data
    last_sequence = data['Temp'].values[-sequence_length:]
    last_sequence = last_sequence.reshape((1, sequence_length, 1))
    
    # Predict the temperature for each day up to the input date
    num_days = (input_date - last_date).days
    predicted_temps = []
    for _ in range(num_days):
        prediction = model.predict(last_sequence)
        predicted_temps.append(prediction[0, 0])
        
        # Update the sequence with the new prediction
        #last_sequence = np.append(last_sequence[:, 1:], [[prediction]], axis=1)

    # Return the last predicted temperature as the result for the input date
    return predicted_temps[-1] if predicted_temps else None

# Example: Predict temperature for a future date
predicted_temp = predict_temp_for_date("1994-09-07", data, model)
print(f"Predicted temperature for 1991-01-01: {predicted_temp}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━

In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Function to predict a given number of future temperatures
def predict_future_temps(data, model, num_days=100, sequence_length=3):
    # Get the last date and last sequence from the data
    last_date = data.index[-1]
    last_sequence = data['Temp'].values[-sequence_length:]
    last_sequence = last_sequence.reshape((1, sequence_length, 1))
    
    # Predict temperatures for the specified number of future days
    predicted_temps = []
    dates = []
    for i in range(num_days):
        prediction = model.predict(last_sequence)
        predicted_temps.append(prediction[0, 0])
        
        # Update the sequence with the new prediction
        last_sequence = np.append(last_sequence[:, 1:], [[prediction]], axis=1)
        
        # Append the date for plotting
        dates.append(last_date + pd.Timedelta(days=i+1))

    # Return the predicted temperatures and corresponding dates
    return dates, predicted_temps

# Predict temperatures for 100 days after the last date
future_dates, future_temps = predict_future_temps(data, model, num_days=100)

# Plotting the predicted temperatures
plt.figure(figsize=(12, 6))
plt.plot(future_dates, future_temps, label="Predicted Temperatures", color="blue")
plt.xlabel("Date")
plt.ylabel("Temperature")
plt.title("Predicted Temperatures for 100 Days Beyond Last Record")
plt.legend()
plt.grid(True)
plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/traitlets/config/application.

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import